In [ ]:
import Bio
from Bio import SeqIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time

In [ ]:
genomeData = "/Users/Edward/Downloads/Comp Bio Project Data/ncbi_datasets/ncbi_dataset/data/genomic.fna"
genomeMetaData = ""
referencePath = "/Users/Edward/Downloads/Comp Bio Project Data/ncbi_datasets/ncbi_dataset/data/reference.fna"

In [ ]:
f = list(SeqIO.parse(genomeData, "fasta"))

In [ ]:
reference = next(SeqIO.parse(referencePath, "fasta"))
reference

In [ ]:
first10 = f[:10]

In [ ]:
list(map(Counter, first10))

In [ ]:
list(map(len, first10))

In [ ]:
def timeFunction(message, f):
    timeStart = time.time()
    f()
    timeEnd = time.time()
    took = round(timeEnd-timeStart, 2)
    print(message(took))
    
# Test case
timeFunction(lambda t: f"Took {t} seconds", lambda: [1+1 for _ in range(10000000)])

In [ ]:
# SeqIO.write(subset, "first10.fna", "fasta")

In [ ]:
# from io import StringIO
# handle = StringIO()
# SeqIO.write(subset, handle, "fasta")
# data = handle.getvalue()

In [ ]:
# from Bio.Align.Applications import ClustalOmegaCommandline as COCL
# help(COCL)
# in_file = "first10.fna"
# out_file = "aligned.fasta"
# co_cline = COCL(infile=in_file, outfile=out_file, verbose=True, auto=True)
# print(co_cline)
# stdout, stderr=co_cline()
# stdout

In [ ]:
from Bio import Align
aligner = Align.PairwiseAligner()

In [ ]:
# https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/EMBOSS+Needle+Help+and+Documentation
# There is apparently no consensus on what the best DNA scoring matrix is. 
# DNAFull is apparently a common one. +5 for match, -4 for mismatch, -10 for gap opening, -0.5 for gap extension
# http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Apairwise-affine-gapscores
# See 6.6.2.5, table 6.1
# You can modify the scores for the upper and lower sequences individually. May be useful.

from Bio.Align import substitution_matrices as sm
# get possible names via sm.load(). Alternatively see github: https://github.com/biopython/biopython/tree/master/Bio/Align/substitution_matrices/data

# We use a modified version of DNAFull, with extension penalties of -2. This is to ensure that gaps aren't favored 
# over matching Ns. 

aligner.substitution_matrix = sm.load("NUC.4.4")
aligner.open_gap_score = -10
aligner.extend_gap_score = -2

In [ ]:
def align(dna):
    output = format(aligner.align(reference.seq, dna)[0])
    l = len(output)//3
    referenceAligned = output[:l-1]
    dnaAligned = output[2*l:3*l-1]
    return (referenceAligned, dnaAligned)

In [ ]:
def getInsertions(refAligned, dnaAligned):
    indels = []
    while "-" in refAligned:
        start = refAligned.index("-")
        end = start
        while end < len(refAligned) and refAligned[end] == "-":
            end += 1
        refAligned = refAligned[:start]+refAligned[end:]
        insert = dnaAligned[start:end]
        dnaAligned = dnaAligned[:start]+dnaAligned[end:]
        indels.append((start, insert))
    return (refAligned, dnaAligned, indels)

In [ ]:
def getDeletions(refAligned, dnaAligned):
    indels = []
    while "-" in dnaAligned:
        start = dnaAligned.index("-")
        end = start
        while end < len(dnaAligned) and dnaAligned[end] == "-":
            end += 1
        insert = refAligned[start:end]
        dnaAligned = dnaAligned[:start]+insert+dnaAligned[end:]
        indels.append((start, len(insert)))
    return (dnaAligned, indels)

In [ ]:
assert(getDeletions("AAATTTGGG", "AAA---GGG") == ("AAATTTGGG", [(3, 3)]))

In [ ]:
def replaceNs(refAligned, dnaAligned):
    indels = []
    while "N" in dnaAligned:
        start = dnaAligned.index("N")
        end = start
        while end < len(dnaAligned) and dnaAligned[end] == "N":
            end += 1
        insert = refAligned[start:end]
        dnaAligned = dnaAligned[:start]+insert+dnaAligned[end:]
        indels.append((start, len(insert)))
    return (dnaAligned, indels)

In [ ]:
assert(replaceNs("AAATTTGGG", "AAANNNGGG") == ("AAATTTGGG", [(3, 3)]))

In [ ]:
def replaceOthers(refAligned, dnaAligned):
    indels = []
    dnaAligned = list(dnaAligned)
    for i in range(len(dnaAligned)):
        if dnaAligned[i] not in ["A", "C", "T", "G"]:
            indels.append((i, dnaAligned[i]))
            dnaAligned[i] = refAligned[i]
    dnaAligned = "".join(dnaAligned)
    return (dnaAligned, indels)

In [ ]:
def getMismatches(refAligned, dnaAligned):
    mismatches = []
    for i in range(len(dnaAligned)):
        if dnaAligned[i] != refAligned[i]:
            mismatches.append((i, dnaAligned[i]))
    return mismatches

assert(getMismatches("AAACCC", "ATACTC") == [(1, "T"), (4, "T")])

In [ ]:
assert(replaceOthers("AAACCC", "AYACMC") == ("AAACCC", [(1, "Y"), (4, "M")]))

In [ ]:
data = f[:100]

In [ ]:
alignedData = map(lambda x: align(reference, x), data)

In [ ]:
def main(n):
    data = f[:n]
    def transform(x):
        refAligned, dnaAligned = align(x.seq)
        refAligned, dnaAligned, insertions = getInsertions(refAligned, dnaAligned)
        dnaAligned, deletions = getDeletions(refAligned, dnaAligned)
        dnaAligned, ns = replaceNs(refAligned, dnaAligned)
        dnaAligned, others = replaceOthers(refAligned, dnaAligned)
        mismatches = getMismatches(refAligned, dnaAligned)
        return (insertions, deletions, ns, others, mismatches)
    return list(map(transform, data))

In [ ]:
r = main()

In [ ]:
r

In [ ]:
def redoInsertions(dna, insertions):
    if insertions == []:
        return dna
    else:
        (position, s) = insertions.pop()
        before, after = dna[:position], dna[position:]
        rec = redoInsertions(before, insertions)
        return rec+list(s)+after

In [ ]:
assert(redoInsertions(list("ACAACA---"), [(3, "AAA")]) == list("ACAAAAACA---"))

In [ ]:
first = r[0]

In [ ]:
def decodeString(reference, insertions, deletions, ns, others, mismatches):
    reference = list(reference)
    dna = list(reference)
    # Insert the mismatches
    for (i, l) in mismatches:
        dna[i] = l
    # Replace the strange characters
    for (i, l) in others:
        dna[i] = l
    # Replace the Ns
    for (startingPosition, length) in ns:
        dna[startingPosition:startingPosition+length] = ["N" for _ in range(length)]
    # Delete the inserted parts
    for (startingPosition, length) in deletions:
        dna[startingPosition:startingPosition+length] = ["-" for _ in range(length)]
    # Fix the insertions
    dna = redoInsertions(dna, insertions)
    # Now remove the "-"s. 
    dna = filter(lambda x: x != "-", dna)
    # Done : ). 
    return "".join(dna)

In [ ]:
str(f[9].seq) == decodeString(reference, *r[9])

In [ ]:
for x in r:
    print(f"Insertions: {len(x[0])}, Deletions: {len(x[1])}, Ns: {len(x[2])}, Others: {len(x[3])}, Mismatches: {len(x[4])}")

In [ ]:
rLs = list(map(lambda x: sum([len(y) for y in x]), r))

In [ ]:
sum(rLs)

In [ ]:
insertions = list(map(lambda x: x[0], r))

In [ ]:
insertions

In [ ]:
r[0]

In [ ]:
charDict = {"A": "00", "C": "01", "T":"10", "G":"11"}
inverseDict = {v: k for k, v in charDict.items()}

# char -> binaryString
def encodeChar(c):
    return charDict[c]

# binaryString -> (char, rest of the string)
def decodeChar(binaryString):
    b = binaryString[:2]
    return (inverseDict[b], binaryString[2:])

# Test cases
# Unecessary

# Input: (number, how many bits to use), 
# Output: The number in binary format, with leading 0s such that length == bits
def encodeNumber(n, bits):
    return bin(n)[2:].rjust(bits, "0")

# Input: binary string * the number of bits used for number, output (position, rest)
def decodeNumber(binaryString, bits):
    b = binaryString[:bits]
    return (int(b, 2), binaryString[bits:])

In [ ]:
def encodeInsertions(insertions):
    # TODO FIX this is a hyperparameter based on an upper length of 2^15 = 32000 (suitable for COVID)
    bits = 15
    eN = lambda n: encodeNumber(n, bits)
    s = ""
    if len(insertions) > 0:
        s += "1"
        s += eN(len(insertions)) # Number of insertions
        for (position, insert) in insertions:
            s += eN(position)
            s += eN(len(insert))
            # Note: This next step assumes that the insert is composed of ACTG. I just realised that because I called getInsertions before I did getOthers,
            # this may not necessarily be true.
            # FIX / TODO
            insertInBinary = "".join(map(lambda x: encodeChar(x), list(insert)))
            s += insertInBinary
    else:
        s += "0"
    return s

def encodeDeletions(deletions):
    # TODO FIX this is a hyperparameter based on an upper length of 2^15 = 32000 (suitable for COVID)
    bits = 15
    eN = lambda n: encodeNumber(n, bits)
    s = ""
    if len(deletions) > 0:
        s += "1"
        s += eN(len(deletions)) # Number of deletions
        for (position, length) in deletions:
            s += eN(position)
            s += eN(length)
    else:
        s += "0"
    return s

def encodeNs(ns):
    # TODO FIX this is a hyperparameter based on an upper length of 2^15 = 32000 (suitable for COVID)
    bits = 15
    eN = lambda n: encodeNumber(n, bits)
    s = ""
    if len(ns) > 0:
        s += "1"
        s += eN(len(ns)) # Number of insertions
        for (position, length) in ns:
            s += eN(position)
            s += eN(length)
    else:
        s += "0"
    return s

def encodeOthers(others):
    # TODO FIX this is a hyperparameter based on an upper length of 2^15 = 32000 (suitable for COVID)
    bits = 15
    eN = lambda n: encodeNumber(n, bits)
    s = ""
    if len(others) > 0:
        s += "1"
        s += eN(len(others)) # Number of insertions
        for (position, c) in others:
            s += eN(position)
            s += encodeNumber(ord(c), 8) # i.e. encode Y in ascii binary (can do this more efficiently but don't think it matters)
    else:
        s += "0"
    return s

def encodeMismatches(mismatches):
    # TODO FIX this is a hyperparameter based on an upper length of 2^15 = 32000 (suitable for COVID)
    bits = 15
    eN = lambda n: encodeNumber(n, bits)
    s = ""
    if len(mismatches) > 0:
        s += "1"
        s += eN(len(mismatches)) # Number of insertions
        for (position, c) in mismatches:
            s += eN(position)
            s += encodeChar(c)
    else:
        s += "0"
    return s

In [ ]:
encodeInfo(r[0])

In [ ]:
len(encodeInfo(r[0]))

In [ ]:
sum(list(map(lambda x: len(encodeInfo(x)), r)))

In [ ]:
sum(map(len, f[:100]))

In [ ]:
28465/(2957317*8)

In [ ]:
1/0.001203159823583336

In [ ]:
list(map(lambda x: str(x[4]), r))

In [ ]:
def flatten(regular_list):
    return [item for sublist in regular_list for item in sublist]

In [ ]:
mismatchPositions = list(map(lambda x: x[0], flatten(list(map(lambda x: x[4], r)))))

In [ ]:
freq = sorted(list(Counter(mismatchPositions).values()))

In [ ]:
len(freq)

In [ ]:
sum(freq)

In [ ]:
freq[-40:]

In [ ]:
%prun aligner.align(reference.seq+reference.seq, f[0].seq+f[0].seq)
# %prun main(1)

In [ ]:
result = aligner.align(reference.seq+reference.seq, f[0].seq+f[0].seq)

In [ ]:
len(result)

In [ ]:
print("hello")

In [ ]:
print("hello")

In [ ]:
from Bio import pairwise2
pairwise2.align.globalds("ACCCCA", "ACCCCCC", aligner.substitution_matrix, -10, -2, one_alignment_only=True)

In [ ]:
b = bytes(b'01101')

In [ ]:
type(b)

In [ ]:
type(bin(98))

In [ ]:
import io
io.BytesIO

In [ ]:
f = open("ZZZZtempfile.bin", "wb")

In [ ]:
f.write(bytes(1))

In [ ]:
f.close()

In [ ]:
f = open("ZZZZtempfile.bin", "rb")

In [ ]:
b = f.read()

In [ ]:
b

In [ ]:
len(b"11")

In [ ]:
int("111", 2)

In [ ]:
5.to_bytes(1, byteorder="big", signed=False)

In [ ]:
len(bytes(b'\b00101011\x02'))

In [ ]:
l = range(1000000)
%timeit list(map(lambda x: x**2, l))

In [ ]:
from multiprocessing import Pool
import math

l = list(range(5))

def square(x):
    l = list(range(500))
    for i in range(len(l)):
        l[i] = i**2
    return sum(l)

# list(map(square, l))

# with Pool() as P:
#     P.map(square, l)

def f(x):
    print("f called")
    for i in range(10):
        x = x * x
    return x

In [ ]:
if __name__ == "__main__":
    print("Hello")
    Pool().map(f, [1, 2, 3])

In [ ]:
getInfoFromString

In [ ]:
main()